In [30]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import requests

class ForumPost:
    
    # CLASS VARIABLE
    num_of_post = 0
    
    def __init__(self, title, content, date, replies):
        self.title = title
        self.content = content
        self.date = date
        self.replies = replies
        
        ForumPost.num_of_post += 1
    
    @property
    def data(self):
        return self.title, self.content, self.date, self.replies
    
    def __repr__(self):
        return f'ForumPost({self.title}, {self.content}, {self.date}, {self.replies})'

def get_forum_content(tid):
    
    "Webscrape the title, content, date, and replies of a post for a given topic_id(tid)"
    
    all_replies = []
    
    page_no = 1
    while True:
        # Get the webpage
        try:
            html = requests.get(f'https://fgforum.fashionguide.com.tw/post_list?topic_id={tid}&page={page_no}')
        except HTTPError as e:
            # couldn't find the webpage on the server
            print(e)
        except URLError as e:
            # More severe problem: cannot find the server
            print('The server could not be found!')
        except Exception as e:
            # Other unknown problems
            print('Sorry, something went wrong...')
            print(e)
        else:
            # Test if no exception are thrown
            print('Webpage successfully retrieved!')

        # Create a BeautifulSoup Object from the webpage
        soup = BeautifulSoup(html.text, 'lxml')

        # Scrape the post title
        try:
            title = soup.find_all('h1', {'class': 'forum-topic-title'})[0].text
        except IndexError as e:
            print("There's no title! title is set to ''.")
            title = ''
            print(e)

        # Scrape the post content
        try:
            content = soup.find_all('div', {'class': 'forum-topic-body'})[0].text
        except IndexError as e:
            print("There's no content! content is set to ''.")
            content = ''
            print(e)

        # Scrape the post date
        try:
            date = soup.find_all('div', {'class': 'forum-post-date'})[0].text
        except IndexError as e:
            print("There's no date! date is set to ''.")
            date = ''
            print(e)

        # Scrape the comments
        replies = []
        for reply in soup.find_all('div', {'class': 'forum-reply-body'}):
            replies.append(reply.text.strip())

        # break if there's no replies, which means we have reached the end of the comments
        if not replies:
            print('Reached the end of the comments!')
            break
        
        # join the replies in the page together, and append them to all_replies
        all_replies.append('\n'.join(replies))

        # Check for the next page!
        page_no += 1

    title = title.strip()
    content = content.strip()
    date = date.strip()
    # join all_replies from different pages together
    replies = '\n'.join(all_replies).strip()
    
#     print('TITLE: ', title)
#     print('CONTENT: ', content)
#     print('DATE: ', date)
#     print('REPLIES: \n', replies)
    
    return title, content, date, replies

def get_ForumPosts(brand_tids):
    """
    Scrape the posts for the given brand topic_ids (brand_tids);
    Create ForumPost objects out of them.
    Returns a list of ForumPost objects
    """
    all_fps = []
    for tid in brand_tids:
        title, content, date, replies = get_forum_content(tid)
        fp = ForumPost(title, content, date, replies)
#         print(fp)
        all_fps.append(fp)
    
    return all_fps

def write2file(filename, brand_ForumPosts):
    import csv
    
    with open(filename + '.csv', 'w', errors='ignore') as new_file:
        # Create a csv.writer object from the new_file
        csv_writer = csv.writer(new_file)
        
        # Write the headers
        csv_writer.writerow(['title', 'content', 'date', 'replies'])
        
        # Write the data
        for fp in brand_ForumPosts:
            csv_writer.writerow(list(fp.data))
        
def main():
    
    MaxAndCo_tids = [50451, 83452, 50599, 65458, 144859, 85500, 139272, 117781, 75798, 47918, 74335, 114806, 28284, 2468, 80250, 120932, 32609, 32588, 58190,\
                    108197, 139194, 114804, 92295, 23108, 40358, 76971, 81728, 41680, 117960, 127702, 123300, 481927, 96654, 36099, 420911, 21404, 29098, 11168,\
                    106080, 134912, 896740, 92205, 57287, 2933, 18886, 91376, 8895, 135305, 8380, 135400, 56128, 74314, 28461, 106720, 42471, 11678, 36877,\
                    5284, 7648, 10536, 130778, 679538, 820706, 65619, 416378, 727098, 75003, 18169, 5626, 15961, 16599, 797504]
    
    ClubMonaco_tids = [134791, 113152, 154713, 774413, 108923, 75459, 121743, 105345, 4887, 43687, 107667, 137493, 91395, 26903, 119449, 116873,\
                       125320, 121089, 379448, 1234789, 771513, 5961]

    Joseph_tids = [234830, 38860, 907714, 10391, 908181, 552904, 25377, 136768, 89732, 36286, 247163, 97175, 90999, 821929, 126807, 32295, 653512,\
                   1163260, 805535, 786710, 779239, 179785, 724221, 549700, 122569, 6600, 8154918, 537039, 38278, 167683]
    
    Maje_tids = [48868, 132450, 906434, 1290942, 118052, 905954, 906114, 119983, 93570, 912921, 907072, 108113, 137998]
    
    iCB_tids = [113152, 1, 5, 99335, 51723, 123411, 79380, 532, 83480, 272920, 21021, 100896, 123961, 86074, 34365, 136768, 891458, 296008, 116812,\
                54357, 72314, 97916, 1335939, 958603, 1012883, 86164, 108694, 108187, 121501, 1269405, 22688, 108197, 115888, 100529, 9399, 3255,\
                1269947, 120514, 122569, 85194, 66281, 98045, 43280, 134934, 5917, 289, 675106, 27938, 29487, 132404, 57144, 75067, 135484, 4413,\
                24381, 10559, 37196, 122708, 86361, 1223515, 116578, 4451, 116580, 110442, 81265, 128369, 2933, 113017, 113039, 55190, 67481, 81314,\
                9642, 119220, 25533, 332739, 105427, 20437, 968666, 144859, 957405, 57313, 91113, 53737, 74739, 83955, 127995, 905726]
    
    ToryBurch_tids = [992671, 1323220, 1234122, 989951, 802555, 1021073, 134170, 398639, 1020248, 1167424, 145598, 1005428, 83713, 3, 169,\
                      594, 5958, 303, 362]
    
    # Create a list of ForumPost objects
    MaxAndCo_ForumPosts = get_ForumPosts(MaxAndCo_tids)
    ClubMonaco_ForumPosts = get_ForumPosts(ClubMonaco_tids)
    Joseph_ForumPosts = get_ForumPosts(Joseph_tids)
    Maje_ForumPosts = get_ForumPosts(Maje_tids)
    iCB_ForumPosts = get_ForumPosts(iCB_tids)
    ToryBurch_ForumPosts = get_ForumPosts(ToryBurch_tids)
    
    # Write to a new csv file using filename='MAx&Co'
    write2file('Max&Co', MaxAndCo_ForumPosts)
    write2file('ClubMonaco', ClubMonaco_ForumPosts)
    write2file('Joseph', Joseph_ForumPosts)
    write2file('Maje', Maje_ForumPosts)
    write2file('iCB', iCB_ForumPosts)
    write2file('ToryBurch', ToryBurch_ForumPosts)
    
if __name__ == '__main__':
    main()

Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage successfully retrieved!
Webpage 